In [ ]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

tfkl = tf.keras.layers
tfk = tf.keras

In [ ]:
## ORIGINAL DATA LOADER
# Init
seed = 123
data_dir = "/kaggle/input/homework1"

img_height = 96
img_width = 96
batch_size = 3542

# Load data
train_val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,validation_split=0.1,
    subset="training",
    seed=123,
    image_size=(img_height, img_width), 
    batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.1,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Load images and labels
for image_batch, labels_batch in train_val_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

for images, labels in train_val_ds.take(1):
    X_train = images.numpy()
    y_train = labels.numpy()

for images, labels in test_ds.take(1):
    X_val = images.numpy()
    y_val = labels.numpy()

# Class weights
class_weights = class_weight.compute_class_weight(class_weight = "balanced",
                                                  classes = np.unique(y_train),
                                                  y = y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))

# Normalize data
X_train = X_train / 255.
X_val = X_val / 255.

# Convert the label to categorical
y_train = tfk.utils.to_categorical(y_train)
y_val = tfk.utils.to_categorical(y_val)

In [ ]:
# Download and plot InceptionV3 Model
supernet = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(299, 299, 3)
)

# supernet.summary()
# tfk.utils.plot_model(supernet)

In [ ]:
# Add GlobalAveragePooling2D
model_base = supernet
output = model_base.output
output = tfkl.GlobalAveragePooling2D()(output)
model = tfk.Model(model_base.input, output, name="inception_v3")
for layer in model_base.layers:
    layer.trainable = False

supernet = model 

In [ ]:
# Add Top
inputs = tfk.Input(shape=(96, 96, 3)) 
x = tfkl.Resizing(299, 299, interpolation="bicubic")(inputs) 
x = supernet(x)

x = tfkl.Flatten(name='Flattening')(x)

x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    256,
    activation='relu',
    kernel_initializer=tfk.initializers.HeUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)

outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer=tfk.initializers.HeUniform(seed))(x) #GlorotUniform

In [ ]:
# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
tl_model.summary()


# Train the model
tl_history = tl_model.fit(
    x=X_train,
    y=y_train,
    batch_size=128,
    epochs=200,
    class_weight = class_weights,
    validation_data=(X_val, y_val),
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)]
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15, 5))
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15, 5))
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Save the best model
tl_model.save('TransferLearningModel.h5')
del tl_model

In [ ]:
## FINE-TUNING

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel.h5')
# ft_model.summary()

# Set all VGG layers to True
ft_model.get_layer('inception_v3').trainable = True
# for i, layer in enumerate(ft_model.get_layer('inception_v3').layers):
#     print(i, layer.name, layer.trainable)

# Freeze first N layers, e.g., until 2th
for i, layer in enumerate(ft_model.get_layer('inception_v3').layers[:2]):
    layer.trainable = False
# for i, layer in enumerate(ft_model.get_layer('inception_v3').layers):
#     print(i, layer.name, layer.trainable)
# ft_model.summary()

# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x=X_train,
    y=y_train,
    batch_size=128, #256,
    epochs=200,
    class_weight = class_weights,
    validation_data=(X_val, y_val),
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)]
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15, 5))
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15, 5))
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Save Final Model
ft_model.save('FineTuningModel_inceptionV3.h5')